In [1]:
import random
import os
import re
import json
import numpy as np
import pandas as pd
import lightgbm as lgbm
from datetime import datetime
from time import gmtime, strftime
from scipy import stats
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import torch
import lightgbm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import f1_score, mean_absolute_error, mean_squared_error

from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from PIL import Image
# from PIL import ImageFile
# ImageFile.LOAD_TRUNCATED_IMAGES = True
from transformers import ViltProcessor, ViltModel


random_seed = 2020
random.seed(random_seed)
np.random.seed(random_seed)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-26 17:38:34.623594: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-26 17:38:35.880508: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-26 17:38:35.880647: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerr

In [16]:
# from transformers import ViltProcessor

# processor = ViltProcessor.from_pretrained("../model/VilT_VQA")
# from transformers import ViltProcessor, ViltForMaskedLM
# import torch

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model = ViltForMaskedLM.from_pretrained("dandelin/vilt-b32-mlm")
# model.to(device)

In [3]:
from transformers import ViltProcessor, ViltModel
text = "a bunch of cats laying on a couch"
# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# image = Image.open(requests.get(url, stream=True).raw)
image = Image.open('../data/none_picture.jpg')
processor = ViltProcessor.from_pretrained("../model/VilT_flickr")
model = ViltModel.from_pretrained("../model/VilT_flickr")
inputs = processor(image, text, return_tensors='pt')
outputs = model(**inputs)
# outputs.pooler_output

Some weights of the model checkpoint at ../model/VilT_flickr were not used when initializing ViltModel: ['rank_output.weight', 'rank_output.bias']
- This IS expected if you are initializing ViltModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViltModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [35]:
dd = pd.DataFrame(cc.detach().numpy())
dd

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.845939,0.192708,-0.244377,0.187807,-0.209908,-0.02653,0.237982,0.101885,-0.672988,-0.08696,...,-0.902678,-0.086495,-0.002625,-0.302162,-0.163377,0.134137,0.164473,0.030958,0.018105,-0.103024
1,0.845939,0.192708,-0.244377,0.187807,-0.209908,-0.02653,0.237982,0.101885,-0.672988,-0.08696,...,-0.902678,-0.086495,-0.002625,-0.302162,-0.163377,0.134137,0.164473,0.030958,0.018105,-0.103024
2,0.845939,0.192708,-0.244377,0.187807,-0.209908,-0.02653,0.237982,0.101885,-0.672988,-0.08696,...,-0.902678,-0.086495,-0.002625,-0.302162,-0.163377,0.134137,0.164473,0.030958,0.018105,-0.103024


In [17]:
# encoding = processor.feature_extractor(image, return_tensors="pt")
# pixel_values = encoding.pixel_values.to(device)
# print(pixel_values.shape)

In [2]:
train_category = json.load(open('../data/train_all_json/train_category.json', encoding="utf-8"))
train_category_pd = pd.DataFrame(train_category)

train_additional = json.load(open('../data/train_all_json/train_additional.json', encoding="utf-8"))
train_additional_pd = pd.DataFrame(train_additional)

train_tags = json.load(open('../data/train_all_json/train_text.json', encoding="utf-8"))
train_tags_pd = pd.DataFrame(train_tags)

train_temporalspatial = json.load(open('../data/train_all_json/train_temporalspatial.json', encoding="utf-8"))
train_temporalspatial_pd = pd.DataFrame(train_temporalspatial)

train_userdata = json.load(open('../data/train_all_json/train_userdata.json', encoding="utf-8"))
train_userdata_pd = pd.DataFrame(train_userdata)

train_img_pd = pd.read_csv('../data/train_all_json/train_imgfile.txt', header=None)
train_img_pd.columns = ['img']
# train_img_pd['img_file'] = train_img_pd['img'].apply(lambda x: '../data/train_img_url/' + x[6:])
train_img_pd['img_file'] = train_img_pd['img']
# Noting 治理直接用转换过的
# train_img_pd['img_file'] = train_img_pd['img'].apply(lambda x: '../data/train_img_url/' + x[30:]) ## 使用img_url里面的

train_label_pd = pd.read_csv('../data/train_all_json/train_label.txt', header=None)
train_label_pd.columns = ['label']

train_data = train_category_pd.merge(train_additional_pd, on=('Pid', 'Uid'), how='left')
train_data = train_data.merge(train_tags_pd, on=('Pid', 'Uid'), how='left')
train_data = train_data.merge(train_temporalspatial_pd, on=('Pid', 'Uid'), how='left')

train_data =  pd.concat([train_data, train_userdata_pd, train_img_pd, train_label_pd], axis=1)
print(len(train_data))
print(train_data.columns)

305613
Index(['Category', 'Concept', 'Pid', 'Uid', 'Subcategory', 'Mediastatus',
       'Pathalias', 'Ispublic', 'Alltags', 'Mediatype', 'Title', 'Postdate',
       'Longitude', 'Geoaccuracy', 'Latitude', 'photo_firstdatetaken',
       'location_description', 'photo_count', 'ispro', 'user_description',
       'canbuypro', 'timezone_timezone_id', 'photo_firstdate',
       'timezone_offset', 'img', 'img_file', 'label'],
      dtype='object')


In [3]:
test_category = json.load(open('../data/test_all_json/test_category.json', encoding="utf-8"))
test_category_pd = pd.DataFrame(test_category)

test_additional = json.load(open('../data/test_all_json/test_additional.json', encoding="utf-8"))
test_additional_pd = pd.DataFrame(test_additional)

test_tags = json.load(open('../data/test_all_json/test_text.json', encoding="utf-8"))
test_tags_pd = pd.DataFrame(test_tags)

test_temporalspatial = json.load(open('../data/test_all_json/test_temporalspatial.json', encoding="utf-8"))
test_temporalspatial_pd = pd.DataFrame(test_temporalspatial)

test_userdata = json.load(open('../data/test_all_json/test_userdata.json', encoding="utf-8"))
test_userdata_pd = pd.DataFrame(test_userdata)

test_img_pd = pd.read_csv('../data/test_all_json/test_imgfile.txt', header=None)
test_img_pd.columns = ['img']
# test_img_pd['img_file'] = test_img_pd['img']
test_img_pd['img_file'] = test_img_pd['img']

test_data = test_category_pd.merge(test_additional_pd, on=('Pid', 'Uid'), how='left') # 基于PID,UID将两个dataframe合并
test_data = test_data.merge(test_tags_pd, on=('Pid', 'Uid'), how='left')
test_data = test_data.merge(test_temporalspatial_pd, on=('Pid', 'Uid'), how='left')

test_data =  pd.concat([test_data, test_userdata_pd, test_img_pd], axis=1)
test_data['label'] = -1

print(len(test_data))
print(test_data.columns)

180581
Index(['Category', 'Concept', 'Pid', 'Uid', 'Subcategory', 'Mediastatus',
       'Pathalias', 'Ispublic', 'Alltags', 'Mediatype', 'Title', 'Postdate',
       'Longitude', 'Geoaccuracy', 'Latitude', 'photo_firstdatetaken',
       'location_description', 'photo_count', 'ispro', 'user_description',
       'canbuypro', 'timezone_timezone_id', 'photo_firstdate',
       'timezone_offset', 'img', 'img_file', 'label'],
      dtype='object')


In [4]:
def pandas_split_valid_test_dataset(pandas_dataset, valid_ratio=0.1, test_ratio=0.1, shuffle=True):
    
    index = list(range(len(pandas_dataset)))
#     if shuffle:
#         random.shuffle(index)
    
    length = len(pandas_dataset)
    len_valid = int(length * valid_ratio + 0.6)
    len_test = int(length * test_ratio + 0.6)

    train_data = pandas_dataset.loc[index[:-len_test-len_valid]]
    valid_data = pandas_dataset.loc[index[-len_test-len_valid:-len_test]]
    test_data = pandas_dataset.loc[index[-len_test:]]
    return train_data, valid_data, test_data

train_df, valid_df, test_df = pandas_split_valid_test_dataset(train_data, valid_ratio=0.1, test_ratio=0.1, shuffle=True)
print(len(train_df), len(valid_df), len(test_df))


train_df['train_type'] = 0
valid_df['train_type'] = 1
test_df['train_type'] = 2
test_data['train_type'] = -1

all_data = pd.concat([train_df, valid_df, test_df, test_data], axis=0, sort=False)
all_data = all_data.reset_index(drop=True)
# print(len(all_data))
# all_data = all_data.fillna('0')

all_data.to_csv('../data/combine_data_530.csv', header=True)


244491 30561 30561


In [2]:
all_data = pd.read_csv('../data/combine_data_530.csv', low_memory=False, index_col=0)
all_data = all_data.fillna('0')
all_data

,Category,Concept,Pid,Uid,Subcategory,Mediastatus,Pathalias,Ispublic,Alltags,Mediatype,...,ispro,user_description,canbuypro,timezone_timezone_id,photo_firstdate,timezone_offset,img,img_file,label,train_type
0,Fashion,glam,775,59@N75,Fashion,ready,luisdrayton,1,rock punk transgender tranny electronicmusic e...,photo,...,1,"0.0866962,-0.0752717,-0.0149407,-0.0460858,-0....",0,9,1213743830,1,train/59@N75/775.jpg,train/59@N75/775.jpg,11.18,0
1,Travel&Active&Sports,fifa,1075,1@N18,Soccer,ready,claudiolara,1,brazil rio brasil riodejaneiro by maria fifa m...,photo,...,1,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743,0,train/1@N18/1075.jpg,train/1@N18/1075.jpg,15.15,0
2,Entertainment,cinema,4890,351@N64,Movies,ready,rizzato,1,old cinema beauty marilyn photoshop movie joke...,photo,...,0,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,11,1296417600,-3,train/351@N64/4890.jpg,train/351@N64/4890.jpg,10.99,0
3,Holiday&Celebrations,old,6568,6@N59,Birthday,ready,4kleuren,1,pictures old family scans brothers sister 1958...,photo,...,1,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743,0,train/6@N59/6568.jpg,train/6@N59/6568.jpg,8.63,0
4,Food,thirsty,7079,1617@N40,Drinks,ready,glosackmd,1,hot sahara animal animals desert bottles drink...,photo,...,1,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743,0,train/1617@N40/7079.jpg,train/1617@N40/7079.jpg,11.16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486189,Travel&Active&Sports,sea,1118324,49686@N77,Seattle,ready,larsgerrits,1,natur nature frog sea lake fish outdoor macro ...,photo,...,1,"0.150263,0.00344591,-0.158953,0.118596,-0.0566...",0,8,1394046082,0,test/49686@N77/1118324.jpg,test/49686@N77/1118324.jpg,-1.00,-1
486190,Travel&Active&Sports,cars,1118329,31147@N75,Cars,ready,None,1,11 arisk okruh veternov 2016 10 jn preov veter...,photo,...,0,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,2,1460319331,0,test/31147@N75/1118329.jpg,test/31147@N75/1118329.jpg,-1.00,-1
486191,Animal,cat,1118336,5656@N26,Cats,ready,rogersg,1,holland netherlands amsterdam cat nl ramses gu...,photo,...,0,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,9,1464117797,1,test/5656@N26/1118336.jpg,test/5656@N26/1118336.jpg,-1.00,-1
486192,Whether&Season,white,1118349,28302@N36,Snowing,ready,ripensis,1,street people bw white black netherlands monoc...,photo,...,1,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,2,1246915883,0,test/28302@N36/1118349.jpg,test/28302@N36/1118349.jpg,-1.00,-1


In [3]:
all_data.columns

Index(['Category', 'Concept', 'Pid', 'Uid', 'Subcategory', 'Mediastatus',
       'Pathalias', 'Ispublic', 'Alltags', 'Mediatype', 'Title', 'Postdate',
       'Longitude', 'Geoaccuracy', 'Latitude', 'photo_firstdatetaken',
       'location_description', 'photo_count', 'ispro', 'user_description',
       'canbuypro', 'timezone_timezone_id', 'photo_firstdate',
       'timezone_offset', 'img', 'img_file', 'label', 'train_type'],
      dtype='object')

In [9]:
all_data['all_text'] = all_data['Title'] + all_data['Alltags']
all_data

,Category,Concept,Pid,Uid,Subcategory,Mediastatus,Pathalias,Ispublic,Alltags,Mediatype,...,train_type,totalViews,totalTags,totalGeotagged,totalFaves,totalInGroup,photoCount,followerCount,followingCount,all_text
0,Fashion,glam,775,59@N75,Fashion,ready,luisdrayton,1,rock punk transgender tranny electronicmusic e...,photo,...,0,6217034,2215,0,904,176,1446,2405,2729,Luis Drayton - Edinburgh shoot #6rock punk tra...
1,Travel&Active&Sports,fifa,1075,1@N18,Soccer,ready,claudiolara,1,brazil rio brasil riodejaneiro by maria fifa m...,photo,...,0,55841091,3991,542,6200,679,75883,8491,236,Arena da Barra - Arena HSBC - Arena do PAN #...
2,Entertainment,cinema,4890,351@N64,Movies,ready,rizzato,1,old cinema beauty marilyn photoshop movie joke...,photo,...,0,5324029,2930,1,2583,2,1039,1550,1346,MARILYN 2015old cinema beauty marilyn photosho...
3,Holiday&Celebrations,old,6568,6@N59,Birthday,ready,4kleuren,1,pictures old family scans brothers sister 1958...,photo,...,0,18358899,5707,716,76088,385,124017,1233,1115,Knikkertijd - 1959pictures old family scans br...
4,Food,thirsty,7079,1617@N40,Drinks,ready,glosackmd,1,hot sahara animal animals desert bottles drink...,photo,...,0,48863537,5344,12,0,102,9218,5781,13,CAMELS01hot sahara animal animals desert bottl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486189,Travel&Active&Sports,sea,1118324,49686@N77,Seattle,ready,larsgerrits,1,natur nature frog sea lake fish outdoor macro ...,photo,...,-1,289375,987,40,4341,84,347,489,612,Quaak!natur nature frog sea lake fish outdoor ...
486190,Travel&Active&Sports,cars,1118329,31147@N75,Cars,ready,None,1,11 arisk okruh veternov 2016 10 jn preov veter...,photo,...,-1,0,0,0,0,0,0,0,0,arisk okruh veternov 201611 arisk okruh vetern...
486191,Animal,cat,1118336,5656@N26,Cats,ready,rogersg,1,holland netherlands amsterdam cat nl ramses gu...,photo,...,-1,10561980,21861,6422,5225,98,6556,702,113,Bike & Cat / Eerste Anjeliers Dwarsstraatholla...
486192,Whether&Season,white,1118349,28302@N36,Snowing,ready,ripensis,1,street people bw white black netherlands monoc...,photo,...,-1,5303389,5677,741,24679,373,2138,1092,389,The Stairs crewstreet people bw white black ne...


In [3]:
wv_model = KeyedVectors.load_word2vec_format('../data/word2vec.txt')
print('load over')

load over


In [7]:

Alltags_split = all_data['Alltags'].apply(lambda x: x.lower().split(' ')) # 文字描述转化为word2vec
# Alltags_split = all_data['Alltags'].apply(lambda x: [w for w in re.sub('[^0-9a-zA-Z]', " ", x).lower().split(' ') if w != ""])

tags_ans = []
for sentence in Alltags_split:
    v = [wv_model[w] for w in sentence if w in wv_model]
    if len(v) == 0:
        tags_ans.append(np.zeros(300))
    else:
        tags_ans.append(np.mean(v, 0))

alltags_feature = np.array(tags_ans)

pd_alltags_feature = pd.DataFrame(alltags_feature, dtype='float')
pd_alltags_feature.columns = ['alltags_fe_{}'.format(i) for i in range(300)]
pd_alltags_feature.to_csv('../data/alltags_feature.csv', header=True, index=None)

print('alltag over!')

alltag over!


In [8]:
Title_split = all_data['Title'].apply(lambda x: x.lower().split(' ')) ## title转化为word2vec
# Title_split = all_data['Title'].apply(lambda x: [w for w in re.sub('[^0-9a-zA-Z]', " ", x).lower().split(' ') if w != ""])

title_ans = []
for sentence in Title_split:
    v = [wv_model[w] for w in sentence if w in wv_model]
    if len(v) == 0:
        title_ans.append(np.zeros(300))
    else:
        title_ans.append(np.mean(v, 0))

title_feature = np.array(title_ans)

pd_title_feature = pd.DataFrame(title_feature, dtype='float')
pd_title_feature.columns = ['title_fe_{}'.format(i) for i in range(300)]
pd_title_feature.to_csv('../data/title_feature.csv', header=True, index=None)

print('title over!')

title over!


In [3]:
# all_data = pd.read_csv('../data/combine_data_530.csv', low_memory=False)
# all_data = all_data.fillna('0')
user_path = dict()
for i in range(len(all_data)):
    user = all_data['Uid'][i]
    path = all_data['Pathalias'][i]
    
    if user not in user_path:
        user_path[user] = set()
    if path != 'None':
        user_path[user].add(path)

Pathalias_list = []
for i in range(len(all_data)):
    user = all_data['Uid'][i]
    if len(user_path[user]) != 0:
        Pathalias_list.append(list(user_path[user])[0])
    else:
        Pathalias_list.append('None')
all_data['Pathalias'] = Pathalias_list


In [4]:

user_additional = pd.read_csv('../data/user_additional.csv', index_col=0)
user_additional[user_additional['Pathalias'] == 'None'] = ['None', 0, 0, 0, 0, 0, 0, 0, 0]

all_data = pd.merge(all_data, user_additional, on='Pathalias', how='left')
all_data['label'] = all_data['label'].apply(lambda x: x if x!=-1 else 0)

In [5]:
all_data['img_file']

0                train/59@N75/775.jpg
1                train/1@N18/1075.jpg
2              train/351@N64/4890.jpg
3                train/6@N59/6568.jpg
4             train/1617@N40/7079.jpg
                     ...             
486189     test/49686@N77/1118324.jpg
486190     test/31147@N75/1118329.jpg
486191      test/5656@N26/1118336.jpg
486192     test/28302@N36/1118349.jpg
486193    test/102192@N21/1118356.jpg
Name: img_file, Length: 486194, dtype: object

In [7]:
len(all_data['img_file'])

486194

In [6]:
from tqdm import trange
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
def get_img_data(img_file):
    if os.path.exists('../data/' + img_file) == True:
        return '../data/' + img_file
    else:
        return '../data/none_picture.jpg'
    
# def get_img_data(img_file):
#     if os.path.exists(img_file) == True:
#         return  img_file
#     else:
#         return '../data/none_picture.jpg'

def get_feature(data_df):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    processor = ViltProcessor.from_pretrained("../model/VilT_flickr", truncation = True, padding = True)
    model = ViltModel.from_pretrained("../model/VilT_flickr")
    model.to(device)
    feature_data = pd.DataFrame()
    feature_data['Pid'] = data_df['Pid']
    feature_data['train_type'] = data_df['train_type']
    
    Uid_set=set(data_df['Uid'])
    Uid_map = dict(zip(Uid_set, list(range(len(Uid_set)))))
    feature_data['Uid'] = data_df['Uid'].map(Uid_map)
    
    feature_data['Uid_count'] = data_df['Uid'].map(dict(data_df.groupby('Uid')['Pid'].count()))
    feature_data['mean_label']= data_df['Uid'].map(dict(data_df.groupby('Uid')['label'].mean()))
    
    # Category
    Category_set=set(data_df['Category'])
    Category_map = dict(zip(Category_set, list(range(len(Category_set)))))
    feature_data['Category'] = data_df['Category'].map(Category_map)

    Subcategory_set=set(data_df['Subcategory'])
    Subcategory_map = dict(zip(Subcategory_set, list(range(len(Subcategory_set)))))
    feature_data['Subcategory'] = data_df['Subcategory'].map(Subcategory_map)
    
    Concept_set=set(data_df['Concept'])
    Concept_map = dict(zip(Concept_set, list(range(len(Concept_set)))))
    feature_data['Concept'] = data_df['Concept'].map(Concept_map)
    
    # title alltags base
    feature_data['Title_len'] = data_df['Title'].apply(lambda x: len(x))
    feature_data['Title_number'] = data_df['Title'].apply(lambda x: len(x.lower().split(' ')))
    feature_data['Alltags_len'] = data_df['Alltags'].apply(lambda x: len(x))
    feature_data['Alltags_number'] = data_df['Alltags'].apply(lambda x: len(x.lower().split(' ')))
#################################################################################################
    data_df['img_file'] = data_df['img_file'].apply(lambda x: get_img_data(x))
    with torch.no_grad():
    # img_mode_map = {'P': 0, 'L': 1, 'RGB': 2, 'CMYK': 3, 'RGBA': 4}
        inputs1 = processor(Image.open(data_df['img_file'][0]), data_df['Title'][0] + ' ' + data_df['Alltags'][0], return_tensors='pt', padding=True , truncation=True).to(device)
        outputs1 = model(**inputs1)
        vilt_fea = outputs1.pooler_output
        for index in trange(486193):
            text = data_df['Title'][index + 1] + ', ' + data_df['Alltags'][index + 1]
            try:
                image = Image.open(data_df['img_file'][index + 1])  
                inputs = processor(image, text, return_tensors='pt', padding=True , truncation=True).to(device)
                outputs = model(**inputs)
                vilt_fea1 = outputs.pooler_output
                vilt_fea = torch.concat((vilt_fea,vilt_fea1),0)
            except: 
                image = Image.open('../data/none_picture.jpg')
                inputs = processor(image, text, return_tensors='pt', padding=True , truncation=True).to(device)
                outputs = model(**inputs)
                vilt_fea1 = outputs.pooler_output
                vilt_fea = torch.concat((vilt_fea,vilt_fea1),0)
    vilt_fea_768 = pd.DataFrame(vilt_fea.cpu().detach().numpy())
    vilt_fea_768.columns = ['vilt_fea_{}'.format(i) for i in range(768)]
    feature_data = pd.concat([feature_data, vilt_fea_768], axis=1)
##################################################################################################
    # img base
    # data_df['img_file'] = data_df['img_file'].apply(lambda x: get_img_data(x))
    # img_mode_map = {'P': 0, 'L': 1, 'RGB': 2, 'CMYK': 3, 'RGBA': 4}
    # img_length, img_width, img_pixel, img_model = [], [], [], []
    # for file in data_df['img_file']:
    #     try:
    #         pm = Image.open(file)  
    #     except: 
    #         pm = Image.open('../data/none_picture.jpg')
    #     img_length.append(pm.size[0])
    #     img_width.append(pm.size[1])
    #     img_pixel.append(pm.size[0] * pm.size[1])
    #     img_model.append(img_mode_map[pm.mode])
    # feature_data['img_length'] = img_length
    # feature_data['img_width'] = img_width
    # feature_data['pixel'] = img_pixel
    # feature_data['img_model'] = img_model
    
    # title svd
#     tf_idf_enc_t = TfidfVectorizer(ngram_range=(1, 2))
#     tf_idf_vec_t = tf_idf_enc_t.fit_transform(data_df['Title'])
#     ###############################
#     # mode_svd_t = pd.DataFrame(tf_idf_vec_t)
#     # mode_svd_t.columns = ['svd_mode_t_{}'.format(i) for i in range(len(mode_svd_t))]
#     ########################################
#     svd_enc_t = TruncatedSVD(n_components=20, n_iter=100, random_state=2020)
#     mode_svd_t = svd_enc_t.fit_transform(tf_idf_vec_t)
#     mode_svd_t = pd.DataFrame(mode_svd_t)
#     mode_svd_t.columns = ['svd_mode_t_{}'.format(i) for i in range(20)]
#     feature_data = pd.concat([feature_data, mode_svd_t], axis=1)
    
#     # Tags svd
#     tf_idf_enc = TfidfVectorizer(ngram_range=(1, 2))
#     tf_idf_vec = tf_idf_enc.fit_transform(data_df['Alltags'])
#     ############################
#     # mode_svd = pd.DataFrame(tf_idf_vec)
#     # mode_svd.columns = ['svd_mode_{}'.format(i) for i in range(len(mode_svd))]
#     #####################
#     svd_enc = TruncatedSVD(n_components=20, n_iter=100, random_state=2020)
#     mode_svd = svd_enc.fit_transform(tf_idf_vec)
#     mode_svd = pd.DataFrame(mode_svd)
#     mode_svd.columns = ['svd_mode_{}'.format(i) for i in range(20)]
#     feature_data = pd.concat([feature_data, mode_svd], axis=1)
    
    Mediatype_set=set(data_df['Mediatype'])
    Mediatype_map = dict(zip(Mediatype_set, list(range(len(Mediatype_set)))))
    feature_data['Mediatype'] = data_df['Mediatype'].map(Mediatype_map)
    
    # Temporal-spatial
    data_df['datetime'] = data_df['Postdate'].apply(lambda x: datetime.fromtimestamp(int(x)))
    feature_data['hour'] = data_df['datetime'].apply(lambda x: x.hour)
    feature_data['day'] = data_df['datetime'].apply(lambda x: x.day)
    feature_data['weekday'] = data_df['datetime'].apply(lambda x: x.weekday())
    feature_data['week_hour'] = data_df['datetime'].apply(lambda x: x.weekday() * 7 + x.hour)
    feature_data['year_weekday'] = data_df['datetime'].apply(lambda x: x.isocalendar()[1])
    
    feature_data['Longitude'] = data_df['Longitude'].apply(lambda x: float(x))
    feature_data['Latitude'] = data_df['Latitude'].apply(lambda x: float(x))
    
    feature_data['Geoaccuracy'] = pd.DataFrame(data_df['Geoaccuracy'], dtype='int')
    
    # User data
    feature_data['photo_count'] = pd.DataFrame(data_df['photo_count'], dtype='int')
    feature_data['ispro'] = pd.DataFrame(data_df['ispro'], dtype='int')
    
    user_fe = pd.DataFrame(np.array(list(data_df["user_description"].apply(lambda x: x.split(',')))), dtype='float')
    user_fe.columns = ['user_fe_{}'.format(i) for i in range(399)]
    
    loc_fe =pd.DataFrame(np.array(list(data_df["location_description"].apply(lambda x: x[:-2].split(',') if x[:-2] !='' else ['0.0']*400))), dtype='float')
    loc_fe.columns = ['loc_fe_{}'.format(i) for i in range(400)]
    feature_data = pd.concat([feature_data, user_fe, loc_fe], axis=1)
    
    photo_firstdate = data_df['photo_firstdate'].apply(lambda x: datetime.fromtimestamp(int(x) if x!='None' else 0))
    feature_data['firstdate'] = (data_df['datetime'] - photo_firstdate).apply(lambda x: x.days)
    feature_data['firstweek'] = feature_data['firstdate'] // 7
    feature_data['firstmonth'] = feature_data['firstdate'] // 30
    
    photo_firstdatetaken = data_df['photo_firstdatetaken'].apply(lambda x: datetime.fromtimestamp(int(x)))
    feature_data['firstdatetaken'] = (data_df['datetime'] - photo_firstdatetaken).apply(lambda x: x.days)
    feature_data['firstdatetakenweek'] = feature_data['firstdatetaken'] // 7
    feature_data['firstdatetakenmonth'] = feature_data['firstdatetaken'] // 30

    # Additional
    feature_data['totalViews'] = pd.DataFrame(data_df['totalViews'], dtype='int')
    feature_data['totalTags'] = pd.DataFrame(data_df['totalTags'], dtype='int')
    feature_data['totalGeotagged'] = pd.DataFrame(data_df['totalGeotagged'], dtype='int')
    feature_data['totalFaves'] = pd.DataFrame(data_df['totalFaves'], dtype='int')
    feature_data['totalInGroup'] = pd.DataFrame(data_df['totalInGroup'], dtype='int')
    feature_data['photoCount'] = pd.DataFrame(data_df['photoCount'], dtype='int')
    meanView, meanTags, meanFaves = [], [], []
    for i in range(len(data_df['photoCount'])):
        if data_df['photoCount'][i] == 0:
            meanView.append(0)
            meanTags.append(0)
            meanFaves.append(0)
        else:
            meanView.append(data_df['totalViews'][i] / data_df['photoCount'][i])
            meanTags.append(data_df['totalTags'][i] / data_df['photoCount'][i])
            meanFaves.append(data_df['totalFaves'][i] / data_df['photoCount'][i])
    feature_data['meanView'] = meanView
    feature_data['meanTags'] = meanTags
    feature_data['meanFaves'] = meanFaves
    feature_data['followerCount'] = pd.DataFrame(data_df['followerCount'], dtype='int')
    feature_data['followingCount'] = pd.DataFrame(data_df['followingCount'], dtype='int')
    
    Ispublic_set=set(data_df['Ispublic'])
    Ispublic_map = dict(zip(Ispublic_set, list(range(len(Ispublic_set)))))
    feature_data['Ispublic'] = data_df['Ispublic'].map(Ispublic_map)

    # label
    feature_data['label'] = pd.DataFrame(data_df['label'], dtype='float')
    return feature_data

import torch

# inputs = processor(image, text, return_tensors='pt')
# outputs = model(**inputs)
save_feature_df = get_feature(all_data)
save_feature_df.to_csv('../data/feature_data_vilt_flick.csv', header=True, index=None)
print('feature save!')


Some weights of the model checkpoint at ../model/VilT_flickr were not used when initializing ViltModel: ['rank_output.bias', 'rank_output.weight']
- This IS expected if you are initializing ViltModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViltModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 486193/486193 [3:49:27<00:00, 35.32it/s]


feature save!


In [7]:
fea_data = pd.read_csv('../data/feature_data_vilt_flick.csv')
fea_data

,Pid,train_type,Uid,Uid_count,mean_label,Category,Subcategory,Concept,Title_len,Title_number,...,totalFaves,totalInGroup,photoCount,meanView,meanTags,meanFaves,followerCount,followingCount,Ispublic,label
0,775,0,50349,4,8.690000,3,42,659,33,6,...,904,176,1446,4299.470263,1.531812,0.625173,2405,2729,1,11.18
1,1075,0,29486,13,7.525385,6,69,259,56,13,...,6200,679,75883,735.884072,0.052594,0.081705,8491,236,1,15.15
2,4890,0,42303,1,10.990000,9,44,17,12,2,...,2583,2,1039,5124.185756,2.820019,2.486044,1550,1346,1,10.99
3,6568,0,51015,1,8.630000,5,0,416,18,3,...,76088,385,124017,148.035342,0.046018,0.613529,1233,1115,1,8.63
4,7079,0,18345,31,8.489677,1,26,6,8,1,...,0,102,9218,5300.882729,0.579735,0.000000,5781,13,1,11.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486189,1118324,-1,38530,2,4.380000,6,73,649,6,1,...,4341,84,347,833.933718,2.844380,12.510086,489,612,1,0.00
486190,1118329,-1,52233,7,1.382857,6,45,194,25,4,...,0,0,0,0.000000,0.000000,0.000000,0,0,1,0.00
486191,1118336,-1,7951,9,3.674444,2,61,508,41,7,...,5225,98,6556,1611.040268,3.334503,0.796980,702,113,1,0.00
486192,1118349,-1,41860,17,5.671176,0,19,112,15,3,...,24679,373,2138,2480.537418,2.655285,11.543031,1092,389,1,0.00


In [8]:
fea_data['vilt_fea_1']

0         0.205525
1        -0.091057
2        -0.043653
3         0.034720
4         0.091591
            ...   
486189   -0.215874
486190   -0.128080
486191   -0.384752
486192    0.137337
486193    0.210052
Name: vilt_fea_1, Length: 486194, dtype: float64

In [8]:
Image.open('../data/none_picture.jpg').verify()

In [17]:
!ls

README.md		  get_data_feature.py	train_k_fold_model.py
download_img_and_user.py  run_step2.sh
get_data_feature.ipynb	  test_k_fold_model.py


In [28]:
fea_data = pd.read_csv('../data/feature_data_530.csv')
fea_data

,Pid,train_type,Uid,Uid_count,mean_label,Category,Subcategory,Concept,Title_len,Title_number,...,totalFaves,totalInGroup,photoCount,meanView,meanTags,meanFaves,followerCount,followingCount,Ispublic,label
0,775,0,21950,4,8.690000,8,30,298,33,6,...,904,176,1446,4299.470263,1.531812,0.625173,2405,2729,1,11.18
1,1075,0,55972,13,7.525385,4,32,66,56,13,...,6200,679,75883,735.884072,0.052594,0.081705,8491,236,1,15.15
2,4890,0,50550,1,10.990000,10,2,9,12,2,...,2583,2,1039,5124.185756,2.820019,2.486044,1550,1346,1,10.99
3,6568,0,9529,1,8.630000,2,57,363,18,3,...,76088,385,124017,148.035342,0.046018,0.613529,1233,1115,1,8.63
4,7079,0,57798,31,8.489677,6,4,180,8,1,...,0,102,9218,5300.882729,0.579735,0.000000,5781,13,1,11.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486189,1118324,-1,24629,2,4.380000,4,36,405,6,1,...,4341,84,347,833.933718,2.844380,12.510086,489,612,1,0.00
486190,1118329,-1,20799,7,1.382857,4,44,78,25,4,...,0,0,0,0.000000,0.000000,0.000000,0,0,1,0.00
486191,1118336,-1,52642,9,3.674444,1,39,366,41,7,...,5225,98,6556,1611.040268,3.334503,0.796980,702,113,1,0.00
486192,1118349,-1,16831,17,5.671176,7,42,272,15,3,...,24679,373,2138,2480.537418,2.655285,11.543031,1092,389,1,0.00


In [12]:
fea_data['img_model'].unique()

array([2, 4, 1, 3, 0])

In [13]:
all_data['img_file']

0                ../data/train/59@N75/775.jpg
1                ../data/train/1@N18/1075.jpg
2              ../data/train/351@N64/4890.jpg
3                    ../data/none_picture.jpg
4             ../data/train/1617@N40/7079.jpg
                         ...                 
486189     ../data/test/49686@N77/1118324.jpg
486190     ../data/test/31147@N75/1118329.jpg
486191      ../data/test/5656@N26/1118336.jpg
486192     ../data/test/28302@N36/1118349.jpg
486193    ../data/test/102192@N21/1118356.jpg
Name: img_file, Length: 486194, dtype: object

In [2]:

all_data = pd.read_csv('../data/feature_data_530.csv')


# glove
glove_tags = pd.read_csv('../data/alltags_feature.csv')
glove_title = pd.read_csv('../data/title_feature.csv')
all_data = pd.concat([all_data, glove_tags, glove_title], axis=1)

In [3]:
all_data

,Pid,train_type,Uid,Uid_count,mean_label,Category,Subcategory,Concept,Title_len,Title_number,...,title_fe_290,title_fe_291,title_fe_292,title_fe_293,title_fe_294,title_fe_295,title_fe_296,title_fe_297,title_fe_298,title_fe_299
0,775,0,21894,4,8.690000,5,65,75,33,6,...,-0.055784,0.197610,-0.166620,-0.007064,0.308736,0.138135,0.149598,0.184107,-0.049617,0.011684
1,1075,0,53866,13,7.525385,0,75,139,56,13,...,0.296695,0.087271,0.126468,-0.103056,0.147320,0.193802,0.035001,-0.045986,0.080715,0.078193
2,4890,0,26948,1,10.990000,3,42,480,12,2,...,0.168270,0.003270,0.171345,0.108478,0.349160,-0.396890,-0.205695,0.068778,-0.128545,-0.080683
3,6568,0,355,1,8.630000,10,68,225,18,3,...,0.109710,0.117009,0.245160,-0.306995,0.387836,0.111045,0.157754,-0.086720,0.220020,-0.045470
4,7079,0,315,31,8.489677,2,43,317,8,1,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486189,1118324,-1,46088,2,4.380000,0,35,301,6,1,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
486190,1118329,-1,12280,7,1.382857,0,60,110,25,4,...,0.134555,0.186670,0.330580,0.227405,0.155600,-0.088244,0.144885,-0.199950,0.033038,-0.065841
486191,1118336,-1,43496,9,3.674444,4,28,186,41,7,...,0.156818,-0.107772,0.157098,-0.079490,0.042594,-0.034590,-0.349488,0.016242,-0.172004,-0.012735
486192,1118349,-1,5492,17,5.671176,7,27,54,15,3,...,0.142362,-0.097873,-0.026855,0.195713,-0.101540,0.240027,-0.119417,-0.012023,0.021850,-0.081305


In [8]:
all_data['label']

0         11.18
1         15.15
2         10.99
3          8.63
4         11.16
          ...  
486189     0.00
486190     0.00
486191     0.00
486192     0.00
486193     0.00
Name: label, Length: 486194, dtype: float64

In [10]:
all_data['Pid']

0             775
1            1075
2            4890
3            6568
4            7079
           ...   
486189    1118324
486190    1118329
486191    1118336
486192    1118349
486193    1118356
Name: Pid, Length: 486194, dtype: int64

In [4]:
all_data['train_type'].unique()

array([ 0,  1,  2, -1])

In [5]:
new_all_data = pd.read_csv('../data/feature_data_530.csv',low_memory=False)
new_all_data

,Pid,train_type,Uid,Uid_count,mean_label,Category,Subcategory,Concept,Title_len,Title_number,...,totalFaves,totalInGroup,photoCount,meanView,meanTags,meanFaves,followerCount,followingCount,Ispublic,label
0,775,0,21894,4,8.690000,5,65,75,33,6,...,904,176,1446,4299.470263,1.531812,0.625173,2405,2729,1,11.18
1,1075,0,53866,13,7.525385,0,75,139,56,13,...,6200,679,75883,735.884072,0.052594,0.081705,8491,236,1,15.15
2,4890,0,26948,1,10.990000,3,42,480,12,2,...,2583,2,1039,5124.185756,2.820019,2.486044,1550,1346,1,10.99
3,6568,0,355,1,8.630000,10,68,225,18,3,...,76088,385,124017,148.035342,0.046018,0.613529,1233,1115,1,8.63
4,7079,0,315,31,8.489677,2,43,317,8,1,...,0,102,9218,5300.882729,0.579735,0.000000,5781,13,1,11.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486189,1118324,-1,46088,2,4.380000,0,35,301,6,1,...,4341,84,347,833.933718,2.844380,12.510086,489,612,1,0.00
486190,1118329,-1,12280,7,1.382857,0,60,110,25,4,...,0,0,0,0.000000,0.000000,0.000000,0,0,1,0.00
486191,1118336,-1,43496,9,3.674444,4,28,186,41,7,...,5225,98,6556,1611.040268,3.334503,0.796980,702,113,1,0.00
486192,1118349,-1,5492,17,5.671176,7,27,54,15,3,...,24679,373,2138,2480.537418,2.655285,11.543031,1092,389,1,0.00


In [6]:
feature_columns = ['Pid', 'train_type', 'mean_label','train_type']
feature_columns += ['user_fe_{}'.format(i) for i in range(399)]
feature_columns += ['loc_fe_{}'.format(i) for i in range(400)]
new_all_data = new_all_data.drop(feature_columns, axis=1)
new_all_data

,Uid,Uid_count,Category,Subcategory,Concept,Title_len,Title_number,Alltags_len,Alltags_number,img_length,...,totalFaves,totalInGroup,photoCount,meanView,meanTags,meanFaves,followerCount,followingCount,Ispublic,label
0,21894,4,5,65,75,33,6,128,12,333,...,904,176,1446,4299.470263,1.531812,0.625173,2405,2729,1,11.18
1,53866,13,0,75,139,56,13,800,65,500,...,6200,679,75883,735.884072,0.052594,0.081705,8491,236,1,15.15
2,26948,1,3,42,480,12,2,188,23,500,...,2583,2,1039,5124.185756,2.820019,2.486044,1550,1346,1,10.99
3,355,1,10,68,225,18,3,61,9,500,...,76088,385,124017,148.035342,0.046018,0.613529,1233,1115,1,8.63
4,315,31,2,43,317,8,1,146,19,500,...,0,102,9218,5300.882729,0.579735,0.000000,5781,13,1,11.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486189,46088,2,0,35,301,6,1,78,13,500,...,4341,84,347,833.933718,2.844380,12.510086,489,612,1,0.00
486190,12280,7,0,60,110,25,4,164,23,500,...,0,0,0,0.000000,0.000000,0.000000,0,0,1,0.00
486191,43496,9,4,28,186,41,7,85,9,500,...,5225,98,6556,1611.040268,3.334503,0.796980,702,113,1,0.00
486192,5492,17,7,27,54,15,3,305,35,500,...,24679,373,2138,2480.537418,2.655285,11.543031,1092,389,1,0.00


In [7]:
re_new_all_data = pd.concat([new_all_data,all_data[['img_file','Category', 'Concept', 'Subcategory', 'Alltags', 'Title']]], axis=1)
re_new_all_data

,Uid,Uid_count,Category,Subcategory,Concept,Title_len,Title_number,Alltags_len,Alltags_number,img_length,...,followerCount,followingCount,Ispublic,label,img_file,Category,Concept,Subcategory,Alltags,Title
0,21894,4,5,65,75,33,6,128,12,333,...,2405,2729,1,11.18,train/59@N75/775.jpg,Fashion,glam,Fashion,rock punk transgender tranny electronicmusic e...,Luis Drayton - Edinburgh shoot #6
1,53866,13,0,75,139,56,13,800,65,500,...,8491,236,1,15.15,train/1@N18/1075.jpg,Travel&Active&Sports,fifa,Soccer,brazil rio brasil riodejaneiro by maria fifa m...,Arena da Barra - Arena HSBC - Arena do PAN #...
2,26948,1,3,42,480,12,2,188,23,500,...,1550,1346,1,10.99,train/351@N64/4890.jpg,Entertainment,cinema,Movies,old cinema beauty marilyn photoshop movie joke...,MARILYN 2015
3,355,1,10,68,225,18,3,61,9,500,...,1233,1115,1,8.63,train/6@N59/6568.jpg,Holiday&Celebrations,old,Birthday,pictures old family scans brothers sister 1958...,Knikkertijd - 1959
4,315,31,2,43,317,8,1,146,19,500,...,5781,13,1,11.16,train/1617@N40/7079.jpg,Food,thirsty,Drinks,hot sahara animal animals desert bottles drink...,CAMELS01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486189,46088,2,0,35,301,6,1,78,13,500,...,489,612,1,0.00,test/49686@N77/1118324.jpg,Travel&Active&Sports,sea,Seattle,natur nature frog sea lake fish outdoor macro ...,Quaak!
486190,12280,7,0,60,110,25,4,164,23,500,...,0,0,1,0.00,test/31147@N75/1118329.jpg,Travel&Active&Sports,cars,Cars,11 arisk okruh veternov 2016 10 jn preov veter...,arisk okruh veternov 2016
486191,43496,9,4,28,186,41,7,85,9,500,...,702,113,1,0.00,test/5656@N26/1118336.jpg,Animal,cat,Cats,holland netherlands amsterdam cat nl ramses gu...,Bike & Cat / Eerste Anjeliers Dwarsstraat
486192,5492,17,7,27,54,15,3,305,35,500,...,1092,389,1,0.00,test/28302@N36/1118349.jpg,Whether&Season,white,Snowing,street people bw white black netherlands monoc...,The Stairs crew


In [8]:
re_new_all_data.to_csv('../data/forauto.csv',header=True,index=None)

In [9]:
re_new_all_data['Category']

,Category,Category
0,5,Fashion
1,0,Travel&Active&Sports
2,3,Entertainment
3,10,Holiday&Celebrations
4,2,Food
...,...,...
486189,0,Travel&Active&Sports
486190,0,Travel&Active&Sports
486191,4,Animal
486192,7,Whether&Season
